In [1]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import datetime as dt
from tqdm.notebook import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def get_tweets(concepto, start_date, end_date, location, max_tweets):

	scrap = sntwitter.TwitterSearchScraper(concepto + ' since:' + start_date + ' until:' + end_date + 'location:' + location + ' -filter:retweets')

	tweets_list = {}

	for i, tweet in tqdm(enumerate(scrap.get_items()), total=max_tweets):
		
		tweets_list[i] = {
			'date': tweet.date,
			'id': tweet.id,
			'rawContent': tweet.rawContent,
			'replyCount': tweet.replyCount,
			'retweetCount': tweet.retweetCount,
			'likeCount': tweet.likeCount,
			'quoteCount': tweet.quoteCount,
			'lang': tweet.lang,
			'place': tweet.place,
			'hashtags': tweet.hashtags,
			'mentionedUsers': tweet.mentionedUsers,
			'user_id': tweet.user.id,
			'user_name': tweet.user.username,
			'user_renderedDescription': tweet.user.renderedDescription,
			'user_join_date': tweet.user.created,
			'user_followers': tweet.user.followersCount,
			'user_location': tweet.user.location,
			'user_verified': tweet.user.verified,
			'inReplyToTweetId': str(tweet.inReplyToTweetId)
		}

		if i > max_tweets:
			break
	
	# return tweets_list
	df = pd.DataFrame.from_dict(tweets_list, orient='index')
	return df

In [ ]:
conceptos=['Convencion_Constitucional',
	'Proceso_Constituyente',
	'Nueva_Constitucion',
	'Convencion_Constituyente']

for concepto in conceptos:
	dataset = get_tweets(concepto, '2021-10-01', '2021-12-01', 'chile', 100_000)
	dataset.to_csv('datasets/2021/' + concepto + '_2021_Q4.csv', index=False)
	print(concepto + ': ' + str(len(dataset)))

In [4]:
# REZAGADOS
conceptos=['Convencion_Constitucional',
	'Proceso_Constituyente',
	'Nueva_Constitucion',
	'Convencion_Constituyente']

conceptos = ['Nueva_Constitucion']

for concepto in conceptos:
	dataset = get_tweets(concepto, '2021-06-30', '2021-07-05', 'chile', 100_000)
	dataset.to_csv('datasets/rezagados/' + concepto + '_3.csv', index=False)
	print(concepto + ': ' + str(len(dataset)))

  0%|          | 0/100000 [00:00<?, ?it/s]

Nueva_Constitucion: 11682
